In [ ]:
# learn about cross validation not just test_train_split

# put lemmatization into model pipeline

In [3]:
from IPython.display import clear_output

In [4]:
!pip3 install -r requirements.txt
clear_output()

In [5]:
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
import statsmodels
import re


In [6]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import nltk 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem import PorterStemmer

nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
clear_output()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report

In [8]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
from joblib import Parallel, delayed

In [9]:
class AdvancedParser:
    """ Parses out rss feeds"""

    def __init__(self, start_date='2022-10-01', end_date='2022-11-01', topic_urls_dict = {}, col_names=[]):
        
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        self.topic_urls_dict = topic_urls_dict
        self.all_dfs = []
        self.col_names = col_names
        
        if not self.col_names:
        # These are the common columns I've seen in podcast rss feeds
            self.col_names = ['published_parsed',
                                'id',
                                'links',
                                'summary',
                                'title',
                                'summary_detail',
                                'title_detail',
                                'published',
                                'guidislink']

    def _retrieve_rss_feed(self, url, topic=''):
        xml_data = feedparser.parse(url)
        df = pd.DataFrame(xml_data.entries)
        df = self._ensure_columns(df, self.col_names)
        df['topic'] = topic
        self.all_dfs.append(df)

    @staticmethod
    def _ensure_columns(df, col_names):
        return df[col_names]

    def gather_data(self):
        assert len(self.topic_urls_dict)

        for topic, urls in self.topic_urls_dict.items():
            for url in urls:
                self._retrieve_rss_feed(url, topic)

        final_df = pd.concat(self.all_dfs)
        final_df.reset_index(drop=True, inplace=True)
        return final_df

In [10]:
topic_urls_dict = {'news' : ["http://rss.cnn.com/rss/cnn_topstories.rss",
                                "https://www.huffpost.com/section/front-page/feed?x=1",
                                "https://feeds.simplecast.com/54nAGcIl",
                                "https://feeds.feedburner.com/Monocle24TheGlobalist",
                                "https://www.theguardian.com/news/series/todayinfocus/podcast.xml"],
                'crime' : ["https://feeds.simplecast.com/qm_9xx0g",
                                "https://rss.art19.com/morbid-a-true-crime-podcast",
                                "https://rss.art19.com/erm-mfm",
                                "https://feeds.megaphone.fm/VMP7924981569",
                                "https://www.omnycontent.com/d/playlist/d83f52e4-2455-47f4-982e-ab790120b954/82e70870-d45e-4b4c-8e17-ab8600091b59/e20bfe1d-24c8-4809-b7b0-ab8600091b62/podcast.rss",
                                "https://feeds.megaphone.fm/darknetdiaries",
                                "https://feeds.simplecast.com/GdzgJRQH",
                                "https://feeds.simplecast.com/xl36XBC2",
                                "https://rss.art19.com/dr-death",
                                "https://podcastfeeds.nbcnews.com/HL4TzgYC"],
                'science' : ["https://feeds.simplecast.com/FO6kxYGj",
                                "http://feeds.feedburner.com/radiolab",
                                "https://media.rss.com/fm/feed.xml",
                                "https://omnycontent.com/d/playlist/e73c998e-6e60-432f-8610-ae210140c5b1/6EA152C0-9E3A-45DE-8672-AE2F0056B113/D8936746-9E22-4DBA-B762-AE2F0056B126/podcast.rss",
                                "https://www.thenakedscientists.com/naked_scientists_podcast.xml"],
                'sport' : ["https://media.rss.com/progressivepodcast/feed.xml",
                                "https://media.rss.com/wholeninesports/feed.xml",
                                "https://media.rss.com/moodysportswithdanandzach/feed.xml",
                                "https://media.rss.com/thejosephvorepodcast/feed.xml",
                                "https://media.rss.com/wholeshabang/feed.xml",
                                "https://mcsorleys.barstoolsports.com/feed/pardon-my-take",
                                "https://feeds.megaphone.fm/ESP2298543312",
                                "https://feeds.megaphone.fm/ESP3500611186",
                                "https://feeds.megaphone.fm/ESP3025643506"]}

a_ps = AdvancedParser('2022-10-01', '2022-11-01', topic_urls_dict)

In [11]:
# all_pc_df = a_ps.gather_data()
# This is saving off the data
# all_pc_df.to_pickle('all_podcast_info.pkl')

# Ingesting the pkl file
all_pc_df = pd.read_pickle('all_podcast_info.pkl')

# Getting rid of rows where summary is not populated
all_pc_df.dropna(inplace=True)
all_pc_df.reset_index(inplace=True, drop=True)

In [12]:
#Checking to see if my data is evenly spread
all_pc_df.groupby('topic').count()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink
topic,,,,,,,,,
crime,2424,2424,2424,2424,2424,2424,2424,2424,2424
news,2169,2169,2169,2169,2169,2169,2169,2169,2169
science,3357,3357,3357,3357,3357,3357,3357,3357,3357
sport,2553,2553,2553,2553,2553,2553,2553,2553,2553


In [13]:
# Since the amount for each topic is not the same, I want to balance this to not influence the model. 
all_topics_list = []
for topic, count in all_pc_df.topic.value_counts().to_dict().items():
    all_topics_list.append(all_pc_df[all_pc_df['topic'] == topic])

# Grabbing the small amount to use as our sample amount for down sampling
smallest_topic_amount = min(all_pc_df.topic.value_counts().to_dict().values())

# This is where we will down sampling our data
temp_dfs = []
for temp_df in all_topics_list:
    temp_dfs.append(temp_df.sample(smallest_topic_amount))

fitted_df = pd.concat(temp_dfs)

In [14]:
#Checking to see if my data is evenly spread
fitted_df.groupby('topic').count()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink
topic,,,,,,,,,
crime,2169,2169,2169,2169,2169,2169,2169,2169,2169
news,2169,2169,2169,2169,2169,2169,2169,2169,2169
science,2169,2169,2169,2169,2169,2169,2169,2169,2169
sport,2169,2169,2169,2169,2169,2169,2169,2169,2169


In [15]:
# Keep target (topic) and ntext
cols_to_keep = ['topic', 'summary']
train_df = fitted_df[cols_to_keep].copy()

In [16]:
# This code was found on https://medium.com/p/f2fc1796e8c7
#Lets normalize the data

def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(<.*?>)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df

clean_df = clean_text(train_df, 'summary', 'clean_summary')
clean_df.tail()

,topic,summary,clean_summary
1150,news,"In Hong Kong, hundreds of thousands remain in ...",in hong kong hundreds of thousands remain in t...
214,news,While coming rulings on abortion and guns have...,while coming rulings on abortion and guns have...
2111,news,As Russian soldiers continue to amass near the...,as russian soldiers continue to amass near the...
298,news,This episode details graphic scenes and contai...,this episode details graphic scenes and contai...
1042,news,The New York Times and Siena College conducted...,the new york times and siena college conducted...


In [17]:
# getting rid of stop words

clean_df['clean_summary'] = clean_df['clean_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_df.head()

,topic,summary,clean_summary
5957,science,"<p>Yes, it's listener mail time once more, thi...",yes listener mail time time emails bummer soci...
6647,science,The Science of Smiles: Just try to resist the ...,science smiles try resist power human smile da...
6418,science,Do you believe there's a natural energy that p...,believe theres natural energy permeates univer...
6869,science,"The next time your car breaks down, count the ...",next time car breaks count number vehicles pas...
6645,science,Urine For a Treat: A urine sample reveals a gr...,urine treat urine sample reveals great deal pe...


In [18]:
# tokenize the summaries

clean_df['summary_tokens'] = clean_df['clean_summary'].apply(lambda x: word_tokenize(x))
clean_df.head()

,topic,summary,clean_summary,summary_tokens
5957,science,"<p>Yes, it's listener mail time once more, thi...",yes listener mail time time emails bummer soci...,"[yes, listener, mail, time, time, emails, bumm..."
6647,science,The Science of Smiles: Just try to resist the ...,science smiles try resist power human smile da...,"[science, smiles, try, resist, power, human, s..."
6418,science,Do you believe there's a natural energy that p...,believe theres natural energy permeates univer...,"[believe, theres, natural, energy, permeates, ..."
6869,science,"The next time your car breaks down, count the ...",next time car breaks count number vehicles pas...,"[next, time, car, breaks, count, number, vehic..."
6645,science,Urine For a Treat: A urine sample reveals a gr...,urine treat urine sample reveals great deal pe...,"[urine, treat, urine, sample, reveals, great, ..."


In [19]:
# Stemming the summary words

def word_stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text

clean_df['summary_tokens_stem'] = clean_df['summary_tokens'].apply(lambda x: word_stemmer(x))
clean_df.head()

,topic,summary,clean_summary,summary_tokens,summary_tokens_stem
5957,science,"<p>Yes, it's listener mail time once more, thi...",yes listener mail time time emails bummer soci...,"[yes, listener, mail, time, time, emails, bumm...","[ye, listen, mail, time, time, email, bummer, ..."
6647,science,The Science of Smiles: Just try to resist the ...,science smiles try resist power human smile da...,"[science, smiles, try, resist, power, human, s...","[scienc, smile, tri, resist, power, human, smi..."
6418,science,Do you believe there's a natural energy that p...,believe theres natural energy permeates univer...,"[believe, theres, natural, energy, permeates, ...","[believ, there, natur, energi, permeat, univer..."
6869,science,"The next time your car breaks down, count the ...",next time car breaks count number vehicles pas...,"[next, time, car, breaks, count, number, vehic...","[next, time, car, break, count, number, vehicl..."
6645,science,Urine For a Treat: A urine sample reveals a gr...,urine treat urine sample reveals great deal pe...,"[urine, treat, urine, sample, reveals, great, ...","[urin, treat, urin, sampl, reveal, great, deal..."


In [20]:
# Lemmatize the summary words

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

clean_df['summary_tokens_lemma'] = clean_df['summary_tokens'].apply(lambda x: word_lemmatizer(x))
clean_df.head()

,topic,summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma
5957,science,"<p>Yes, it's listener mail time once more, thi...",yes listener mail time time emails bummer soci...,"[yes, listener, mail, time, time, emails, bumm...","[ye, listen, mail, time, time, email, bummer, ...","[yes, listener, mail, time, time, email, bumme..."
6647,science,The Science of Smiles: Just try to resist the ...,science smiles try resist power human smile da...,"[science, smiles, try, resist, power, human, s...","[scienc, smile, tri, resist, power, human, smi...","[science, smile, try, resist, power, human, sm..."
6418,science,Do you believe there's a natural energy that p...,believe theres natural energy permeates univer...,"[believe, theres, natural, energy, permeates, ...","[believ, there, natur, energi, permeat, univer...","[believe, there, natural, energy, permeates, u..."
6869,science,"The next time your car breaks down, count the ...",next time car breaks count number vehicles pas...,"[next, time, car, breaks, count, number, vehic...","[next, time, car, break, count, number, vehicl...","[next, time, car, break, count, number, vehicl..."
6645,science,Urine For a Treat: A urine sample reveals a gr...,urine treat urine sample reveals great deal pe...,"[urine, treat, urine, sample, reveals, great, ...","[urin, treat, urin, sampl, reveal, great, deal...","[urine, treat, urine, sample, reveals, great, ..."


In [21]:
# This is performing Part of Speech tagging
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text

clean_df['summary_tokens_pos_tagged'] = clean_df['summary_tokens'].apply(lambda x: word_pos_tagger(x))
clean_df.head()

,topic,summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
5957,science,"<p>Yes, it's listener mail time once more, thi...",yes listener mail time time emails bummer soci...,"[yes, listener, mail, time, time, emails, bumm...","[ye, listen, mail, time, time, email, bummer, ...","[yes, listener, mail, time, time, email, bumme...","[(yes, UH), (listener, JJR), (mail, NN), (time..."
6647,science,The Science of Smiles: Just try to resist the ...,science smiles try resist power human smile da...,"[science, smiles, try, resist, power, human, s...","[scienc, smile, tri, resist, power, human, smi...","[science, smile, try, resist, power, human, sm...","[(science, NN), (smiles, NNS), (try, VBP), (re..."
6418,science,Do you believe there's a natural energy that p...,believe theres natural energy permeates univer...,"[believe, theres, natural, energy, permeates, ...","[believ, there, natur, energi, permeat, univer...","[believe, there, natural, energy, permeates, u...","[(believe, NN), (theres, VBZ), (natural, JJ), ..."
6869,science,"The next time your car breaks down, count the ...",next time car breaks count number vehicles pas...,"[next, time, car, breaks, count, number, vehic...","[next, time, car, break, count, number, vehicl...","[next, time, car, break, count, number, vehicl...","[(next, JJ), (time, NN), (car, NN), (breaks, N..."
6645,science,Urine For a Treat: A urine sample reveals a gr...,urine treat urine sample reveals great deal pe...,"[urine, treat, urine, sample, reveals, great, ...","[urin, treat, urin, sampl, reveal, great, deal...","[urine, treat, urine, sample, reveals, great, ...","[(urine, JJ), (treat, NN), (urine, JJ), (sampl..."


In [22]:
# also need to map topics to nums (targets)
topics = clean_df.topic.unique().tolist()
topic_map = dict(zip(topics, list(range(0, len(topics)))))
inverse_topic_map = dict(zip(list(range(0, len(topics))), topics))
clean_df['topic_num'] = clean_df['topic'].apply(lambda x: topic_map.get(x))

In [23]:
# Used https://towardsdatascience.com/how-to-enter-your-first-kaggle-competition-4717e7b232db as resource
# Splitting our data to test and train and creating a model
X_train, X_test, y_train, y_test = train_test_split(clean_df['clean_summary'],clean_df['topic_num'],random_state = 0)

# This is good practice to keep things standardized
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', LogisticRegression()),
])
model = pipeline_sgd.fit(X_train, y_train)

In [24]:
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       537
           1       1.00      0.97      0.99       563
           2       0.97      0.97      0.97       543
           3       0.95      0.98      0.96       526

    accuracy                           0.98      2169
   macro avg       0.98      0.98      0.98      2169
weighted avg       0.98      0.98      0.98      2169



In [25]:
# Let's bring in different podcasts to test our model more

test_topic_urls_dict = {
                        'news' : ["https://feeds.megaphone.fm/slatesthegist"],
                        'crime' : ["https://audioboom.com/channels/4940872.rss",
                                "https://rss.art19.com/generation-why-podcast"],
                        'science' : ["https://feed.theskepticsguide.org/feed/rss.aspx?feed=sgu"], 
                        'sport' : ["https://feeds.megaphone.fm/ESP4820632502"]
}
test_parse = AdvancedParser('2022-10-01', '2022-11-01', test_topic_urls_dict)
test_df = test_parse.gather_data()

In [26]:
# map topics to nums (targets)
test_df['topic_num'] = test_df['topic'].apply(lambda x: topic_map.get(x))

In [48]:
submission_test_clean = test_df.copy()
submission_test_clean = clean_text(submission_test_clean, "summary", "summary_clean")
submission_test_clean['summary'] = submission_test_clean['summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
submission_test_clean.head()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink,topic,topic_num,summary_clean
0,"(2022, 11, 16, 21, 56, 41, 2, 320, 0)",465e9b4e-4e2a-11ed-909a-0f2401518281,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","Gordon Sondland Ambassador EU, eventually led ...",Implicating Trump Under Oath,"{'type': 'text/plain', 'language': None, 'base...","{'type': 'text/plain', 'language': None, 'base...","Wed, 16 Nov 2022 21:56:41 -0000",False,news,3,gordon sondland was ambassador to the eu which...
1,"(2022, 11, 15, 22, 4, 15, 1, 319, 0)",46b09cc8-4e2a-11ed-909a-43b06a29fdea,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",Today test thesis Universal Basic Book Deal (U...,The Democratization Of The Book Deal,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Tue, 15 Nov 2022 22:04:15 -0000",False,news,3,today we test the thesis that a universal basi...
2,"(2022, 11, 14, 23, 22, 0, 0, 318, 0)",46e05288-4e2a-11ed-909a-a7bb01bdf4e1,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",Princess Martha Louise Norway engaged American...,The Princess And The Shaman (Who Might Also Be...,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Mon, 14 Nov 2022 23:22:00 -0000",False,news,3,princess martha louise of norway is engaged to...
3,"(2022, 11, 12, 22, 30, 0, 5, 316, 0)",45be14ee-4e2a-11ed-909a-9bf573d3c5c0,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","In installment Best Of The Gist, look squishie...",BEST OF THE GIST: Squishy Science Edition,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Sat, 12 Nov 2022 22:30:00 -0000",False,news,3,in this installment of best of the gist we loo...
4,"(2022, 11, 11, 22, 0, 0, 4, 315, 0)",460fe80a-4e2a-11ed-909a-fb0f0740e50b,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","The pundit, case, The Gist host Mike Pesca. In...",What Went Right For The Dems And Wrong For The...,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Fri, 11 Nov 2022 22:00:00 -0000",False,news,3,the pundit in this case being the gist host mi...


In [57]:
# Run the model
submission_test_pred = model.predict(submission_test_clean['summary_clean'])
submission_test_clean['prediction'] = submission_test_pred
submission_test_clean['prediction_topic'] = submission_test_clean['prediction'].apply(lambda x: inverse_topic_map.get(x))
submission_test_clean[['topic', 'prediction_topic']][:20]
submission_test_clean.head()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink,topic,topic_num,summary_clean,prediction,prediction_topic,correct_prediction
0,"(2022, 11, 16, 21, 56, 41, 2, 320, 0)",465e9b4e-4e2a-11ed-909a-0f2401518281,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","Gordon Sondland Ambassador EU, eventually led ...",Implicating Trump Under Oath,"{'type': 'text/plain', 'language': None, 'base...","{'type': 'text/plain', 'language': None, 'base...","Wed, 16 Nov 2022 21:56:41 -0000",False,news,3,gordon sondland was ambassador to the eu which...,1,sport,True
1,"(2022, 11, 15, 22, 4, 15, 1, 319, 0)",46b09cc8-4e2a-11ed-909a-43b06a29fdea,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",Today test thesis Universal Basic Book Deal (U...,The Democratization Of The Book Deal,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Tue, 15 Nov 2022 22:04:15 -0000",False,news,3,today we test the thesis that a universal basi...,1,sport,False
2,"(2022, 11, 14, 23, 22, 0, 0, 318, 0)",46e05288-4e2a-11ed-909a-a7bb01bdf4e1,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",Princess Martha Louise Norway engaged American...,The Princess And The Shaman (Who Might Also Be...,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Mon, 14 Nov 2022 23:22:00 -0000",False,news,3,princess martha louise of norway is engaged to...,1,sport,False
3,"(2022, 11, 12, 22, 30, 0, 5, 316, 0)",45be14ee-4e2a-11ed-909a-9bf573d3c5c0,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","In installment Best Of The Gist, look squishie...",BEST OF THE GIST: Squishy Science Edition,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Sat, 12 Nov 2022 22:30:00 -0000",False,news,3,in this installment of best of the gist we loo...,1,sport,False
4,"(2022, 11, 11, 22, 0, 0, 4, 315, 0)",460fe80a-4e2a-11ed-909a-fb0f0740e50b,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","The pundit, case, The Gist host Mike Pesca. In...",What Went Right For The Dems And Wrong For The...,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Fri, 11 Nov 2022 22:00:00 -0000",False,news,3,the pundit in this case being the gist host mi...,1,sport,False


In [58]:
submission_test_clean['correct_prediction'] = submission_test_clean['topic_num'] == submission_test_clean['prediction']

In [59]:
submission_test_clean['correct_prediction'].value_counts()

True     1965
False    1760
Name: correct_prediction, dtype: int64

In [60]:
# Getting the correct numbers out 
prediction_nums = submission_test_clean.groupby('topic_num')['correct_prediction'].value_counts().to_dict()

In [61]:
plot = go.Figure(data=[go.Bar(
    name = 'Correct',
    x = [k[0] for k,v in prediction_nums.items() if k[1]],
    y = [v for k,v in prediction_nums.items() if k[1]]
   ),
                       go.Bar(
    name = 'Wrong',
    x = [k[0] for k,v in prediction_nums.items() if not k[1]],
    y = [v for k,v in prediction_nums.items() if not k[1]]
   )
])

plot.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = submission_test_clean['topic_num'].unique().tolist(),
        ticktext = submission_test_clean['topic'].unique().tolist()
    )
)

                   
plot.show()

In [65]:
# News is the worst by far, but let's see what it is mistaken for
# gather all the prediction values
all_predictions = submission_test_clean.groupby(['topic_num', 'correct_prediction'])['prediction'].value_counts().to_dict()

# put into dataframe for easy plotting
wrong_guesses = pd.DataFrame([{'topic' : k[0], 'incorrect_prediction':k[2], 'amount' : v} for k,v in all_predictions.items() if not k[1]])

#mapping the text for the prediction categories
wrong_guesses['incorrect_prediction_text'] = wrong_guesses['incorrect_prediction'].map(inverse_topic_map)
wrong_guesses['topic_text'] = wrong_guesses['topic'].map(inverse_topic_map)

wrong_plot = px.bar(wrong_guesses, 
    x='topic_text', 
    y='amount',
    text='incorrect_prediction_text', 
    color='incorrect_prediction_text',
    barmode='group',
)
                   
wrong_plot.show()

In [55]:
# If you think about it this makes sense
# News is highly related to multiple subjects
# But science, sports, and crime text is more specific than news (I'm guessing)

In [72]:
submission_test_clean[(~submission_test_clean['correct_prediction']) & 
(submission_test_clean['topic']=='news') &
(submission_test_clean['prediction_topic']=='crime')]['summary_clean'].tolist()[:3]

['ten years ago a disturbed young man entered a colorado movie theater where he shot and killed twelve people wounding  more we speak with his psychiatrist lynne fenton about warning signs treatment options and what more if anything could have been done fentons new book is aurora the psychiatrist who treated the movie theater killer tells her story plus tomorrows  billion lottery will probably make the winner happy which is the point but it seems to be one many people rebut and in the spiel a senate hearing on policing shows the challenges democrats face on a very important issue to votersproduced by joel patterson and corey waraemail us at thegistcomto advertise on the show visit learn more about your ad choices visit megaphonefmadchoices',
 'on the gist the gift to the morehouse graduates is great but should it be celebrated or just the standard in the interview linda taylor committed many crimes but only one of themwelfare fraudreally mattered to politicians the press and the public